# **Example of use of SITS Package - Multiprocessing**

---

We aim to retrieve satellite time series for a set of points in Europe. Rather than processing the points sequentially, we use here the options offered by the `sits.Multiproc()` class to distribute the calculations and thus optimize the processing times.

---

### 1. Installation of SITS package and its depedencies

First, install `sits` package with [pip](https://pypi.org/project/SITS/). We also need some other packages for displaying data.

In [1]:
# SITS package
!pip install --upgrade sits

# other packages
!pip install folium
!pip install mapclassify
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Now we can import `sits` and some other libraries.

In [37]:
import os
# sits lib
import sits
# geospatial libs
import geopandas as gpd
import pandas as pd
# date format
from datetime import datetime
# ignore warnings messages
import warnings
#warnings.filterwarnings('ignore')

import logging
from rasterio.errors import NotGeoreferencedWarning

# Configure logging to ignore the NotGeoreferencedWarning
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

# Now use warnings to filter the NotGeoreferencedWarning
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)

## 2. Handling the input vector file

### 2.1. Data loading

The geojson vector file describing the position of the sandbank is stored in the [Github repository](https://github.com/kenoz/SITS_utils). We download it into our current workspace.  

In [3]:
!mkdir -p test_data
![ ! -f test_data/rand_pts.geojson ] && wget https://raw.githubusercontent.com/kenoz/SITS_utils/refs/heads/main/data/rand_pts.geojson -P test_data

--2025-01-28 11:58:12--  https://raw.githubusercontent.com/kenoz/SITS_utils/refs/heads/main/data/rand_pts.geojson
Resolving proxy.cidsn.jrc.it (proxy.cidsn.jrc.it)... 139.191.240.208
Connecting to proxy.cidsn.jrc.it (proxy.cidsn.jrc.it)|139.191.240.208|:8888... connected.
Proxy request sent, awaiting response... 200 OK
Length: 3872 (3.8K) [text/plain]
Saving to: ‘test_data/rand_pts.geojson’

rand_pts.geojson    100%[===================>]   3.78K  --.-KB/s    in 0s      

2025-01-28 11:58:12 (8.92 MB/s) - ‘test_data/rand_pts.geojson’ saved [3872/3872]



We load the vector file, named `banc_arguin.geojson`, as a geoDataFrame object with the `sits` method: `sits.Vec2gdf()`.

In [4]:
data_dir = 'test_data'
random_pts = sits.Vec2gdf(os.path.join(data_dir, 'rand_pts.geojson'))
random_pts.gdf

,id,pt_id,geometry
0,1,1,POINT (8.49138 49.85437)
1,3,2,POINT (8.41277 53.14555)
2,7,3,POINT (11.17678 50.01380)
3,9,4,POINT (23.79724 40.06894)
4,10,5,POINT (16.80020 48.98809)
5,12,6,POINT (22.49204 42.36324)
6,14,7,POINT (-6.51812 39.32035)
7,16,8,POINT (27.54503 40.23213)
8,19,9,POINT (22.84505 41.47664)
9,21,10,POINT (1.29386 44.31059)


### 2.1. Bounding box calculation

We check the coordinate reference system (CRS) and calculate the bounding box with the method `set_bbox()` of class `sits.Vec2gdf`.

In [26]:
# check epsg
print(f"epsg code for 'random_pts.gdf':  {random_pts.gdf.crs.to_epsg()}")

random_pts.set_buffer('gdf', 0.01)
random_pts.set_bbox('buffer')

epsg code for 'random_pts.gdf':  4326


We display the `sits.Vec2gdf` objects (`.gdf` and _in green_ `.bbox` _in blue_) on an interactive map.

In [27]:
import folium

f = folium.Figure(height=300)
m = folium.Map(location=[45.0, 10], zoom_start=4).add_to(f)
random_pts.gdf.explore(m=m, height=400, color='green')
random_pts.buffer.explore(m=m, height=400)
random_pts.bbox.explore(m=m, height=400, color='red')

### 2.3. CRS management

In order to request data on a STAC catalog, we need to provide the bounding box coordinates in Lat/Long, i.e the EPSG:4326. Then we also need to specify in which CRS we want to obtain the satellite time series. As we are working in France, it can be the EPSG 2154 (RGF-93, Lambert-93) or the EPSG 3035 (ETRS89-extended), valid at the European scale.

Here we calculate the coordinates in EPSG:4326 and EPSG:3035. Since there is only one polygon, we keep the coordinates into two lists.

In [21]:
bbox_4326 = list(random_pts.gdf.iloc[0]['geometry'].bounds)
bbox_3035 = list(random_pts.gdf.to_crs(3035).iloc[0]['geometry'].bounds)

print(f'bbox in EPSG:4326: {bbox_4326}')
print(f'bbox in EPSG:3035: {bbox_3035}')

bbox in EPSG:4326: [8.491383388733258, 49.854370172191935, 8.491383388733258, 49.854370172191935]
bbox in EPSG:3035: [4212506.405420883, 2972428.99783603, 4212506.405420883, 2972428.99783603]


In [28]:
bbox_latlong = pd.concat([random_pts.bbox, random_pts.bbox['geometry'].bounds], axis=1)
bbox_latlong['bbox_4326'] = bbox_latlong[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

bbox_3035 = random_pts.bbox.to_crs(3035)
test_3035_bounds = pd.concat([bbox_3035, bbox_3035['geometry'].bounds], axis=1)
test_3035_bounds['bbox_3035'] = test_3035_bounds[['minx', 'miny', 'maxx', 'maxy']].values.tolist()

test_process = pd.concat([bbox_latlong['bbox_4326'], test_3035_bounds['bbox_3035']], axis=1)
test_process['bbox_tuple'] = test_process.apply(lambda row: (row['bbox_4326'], row['bbox_3035']), axis=1)

test_process.head()

,bbox_4326,bbox_3035,bbox_tuple
0,"[8.481383388733258, 49.84437017219194, 8.50138...","[4211764.670013768, 2971302.3517032275, 421324...","([8.481383388733258, 49.84437017219194, 8.5013..."
1,"[8.402773153496353, 53.135550217099215, 8.4227...","[4214105.200943511, 3337504.924185555, 4215492...","([8.402773153496353, 53.135550217099215, 8.422..."
2,"[11.166778399392497, 50.00380142653234, 11.186...","[4404616.995128865, 2988598.266294405, 4406085...","([11.166778399392497, 50.00380142653234, 11.18..."
3,"[23.78724480625752, 40.05894472525313, 23.8072...","[5495790.682699846, 1992060.233281068, 5497840...","([23.78724480625752, 40.05894472525313, 23.807..."
4,"[16.790197637816156, 48.978094737864616, 16.81...","[4817203.221624014, 2896784.6045746827, 481886...","([16.790197637816156, 48.978094737864616, 16.8..."


In [34]:
%%time

multi = sits.Multiproc('image', 'nc', data_dir)

multi.addParams_stacAttack(bands=['B03', 'B04', 'B08', 'SCL'])
multi.addParams_searchItems(date_start=datetime(2024, 1, 1), 
                            date_end=datetime(2025, 2, 1), 
                            query={"eo:cloud_cover": {"lt": 10}})
multi.addParams_loadCube(resolution=20)
multi.addParams_mask()

for gid, i in enumerate(test_process['bbox_tuple']):
    multi.fetch_func(i[0], i[1], gid, mask=True, gapfill=True)
multi.dask_compute()

/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11

CPU times: user 51.6 ms, sys: 16.9 ms, total: 68.5 ms
Wall time: 1min 29s


(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [36]:
%%time

multi = sits.Multiproc('patch', 'nc', data_dir)

multi.addParams_stacAttack(bands=['B03', 'B04', 'B08', 'SCL'])
multi.addParams_searchItems(date_start=datetime(2024, 1, 1), 
                            date_end=datetime(2025, 2, 1), 
                            query={"eo:cloud_cover": {"lt": 10}})
multi.addParams_loadCube(dimx=10, dimy=10, resolution=20)
multi.addParams_mask()

for gid, i in enumerate(test_process['bbox_tuple']):
    multi.fetch_func(i[0], i[1], gid, mask=True, gapfill=True)
multi.dask_compute()

/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/osekenj/.local/lib/python3.11

CPU times: user 60.9 ms, sys: 28 ms, total: 88.9 ms
Wall time: 1min 33s


(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

## 3. Loading and preprocessing of a Satellite Image Time-Series (SITS)

In this example, we have only one area (one polygon) to process. We use the class `sits.StacAttack` to request and preprocess the data. In case you need to distribute the processing, take a look on the `sits.Multiproc()` approach, using `Dask`.

### 3.1. Creation of a Datacube from STAC catalog

The request consists in retrieving Sentinel-2 images (level 2A) acquired from January 1, 2016 to January 1, 2025 with cloud cover less than 10%. Then we build a 4 bands geo-datacube ('B03', 'B04', 'B08' and 'SCL') in EPSG:3035 with a 20m spatial resolution.

The method `StacAttack.loadCube()` returns an object `StacAttack.cube` i.e. an `xarray.Dataset()`. If necessary, you can modify it using the methods of `xarray.Dataset()` (not really recommended).

### 3.2. Create and apply a mask

We want to mask the defective and cloudy pixels in the Datacube (`StacAttack.cube`). To do this, we use the SCL band provided with the Sentinel-2 images. SCL refers to "Scene Classification Layer" and has been developed to developed to distinguish between cloudy pixels, clear pixels and water pixels. It consists of 12 classes.

In this example we create a mask based on the following classes:
- 0: No Data (Missing data)
- 1: Saturated or defective pixel
- 3: Cloud shadows
- 8: Cloud medium probability
- 9: Cloud high probability
- 10: Thin cirrus



The method `StacAttack.mask()` returns an object `StacAttack.mask` i.e. an `xarray.Dataaarray()`.

We can apply the mask on the Satellite Image Time Series with the method `StacAttack.mask()`.

### 3.3. Gap filling the masked pixels

We interpolate in time the masked pixels (NaN values). The method `StacAttack.gapfill()` relies on the `xarray.DataArray.interpolate_na`. By default the linear interpolation is used but you can try another one (see the related [documentation](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.interpolate_na.html)). As the interpolation needs backward and forward values, the first and the last images are not processed. To avoid this, the method `StacAttack.gapfill()` also call the methods `xarray.DataArray.bfill` and `xarray.DataArray.ffill`. You can disable this by passing the argument `first_last=False`.

## 4. Saving the Datacube as a file

### 4.1. Default export

It is possible to export the Datacube in NetCDF (Network Common Data Form) or in CSV (Comma-Separated Values).


The output filename is automaticaly made with the following syntax:
```sh
fid-<gid>_<array type>_<start date>-<end date>.nc
```


## 5. Convert NetCDF file into animated GIF

Last but not least... sits package allows you to export satellite time series as animated GIF, so you can easily show some phenoma that vary in time and space.

### 5.1. Loading NetCDF file

We load the newly created NetCDF file as an `xarray.Dataarray`, and choose to keep only three spectral bands in order to display color composites (RGB format).

### 5.2. Making a nice-looking animation

To convert the `xarray.Dataarray` object into an animated GIF, `sits` package uses in the geogif library. So it is possible to add specific arguments, not presented by default.

Now it's time to see the result. Enjoy the movie!